In [ ]:
#| default_exp utils

In [ ]:
#| export
import pickle
import torch
from pathlib import Path
import mlflow
import mlflow.pytorch
import os
import tempfile
import json
from fastai.callback.core import Callback
from typing import Any

# Utils
>

In [ ]:
#| export
def store_variables(pkl_fn: str | Path, size: list, reorder: bool, resample: int | list):
    """Save variable values in a pickle file."""
    
    var_vals = [size, reorder, resample]
    
    with open(pkl_fn, 'wb') as f:
        pickle.dump(var_vals, f)

In [ ]:
#| export
def load_variables(pkl_fn: (str, Path)):
    """Loads stored variable values from a pickle file.

    Args:
        pkl_fn: File path of the pickle file to be loaded.

    Returns:
        The deserialized value of the pickled data.
    """
    with open(pkl_fn, 'rb') as f:
        return pickle.load(f)

In [ ]:
#| export
def print_colab_gpu_info(): 
    """Check if we have a GPU attached to the runtime."""
    
    colab_gpu_msg =(f"{'#'*80}\n"
                    "Remember to attach a GPU to your Colab Runtime:"
                    "\n1. From the **Runtime** menu select **Change Runtime Type**"
                    "\n2. Choose **GPU** from the drop-down menu"
                    "\n3. Click **'SAVE'**\n"
                    f"{'#'*80}")
    
    if torch.cuda.is_available(): print('GPU attached.')
    else: print(colab_gpu_msg)

In [ ]:
#| export
class ModelTrackingCallback(Callback):
    """
    A FastAI callback for comprehensive MLflow experiment tracking.
    
    This callback automatically logs hyperparameters, metrics, model artifacts,
    and configuration to MLflow during training.
    """
    
    def __init__(
        self, 
        model_name: str, 
        loss_function: str, 
        item_tfms: list[Any],
        size: list[int], 
        resample: list[float], 
        reorder: bool
    ):
        """
        Initialize the MLflow tracking callback.
        
        Args:
            model_name: Name of the model architecture for registration
            loss_function: Name of the loss function being used
            size: Model input dimensions
            resample: Resampling dimensions
            reorder: Whether reordering augmentation is applied
        """
        self.model_name = model_name
        self.loss_function = loss_function
        self.item_tfms = item_tfms
        self.size = size
        self.resample = resample
        self.reorder = reorder
        
        self.config = self._build_config()
        
    def extract_all_params(self, tfm):
        """
        Extract all parameters from a transform object for detailed logging.
        
        Args:
            tfm: Transform object to extract parameters from
            
        Returns:
            dict: Dictionary with 'name' and 'params' keys containing transform details
        """
        class_name = tfm.__class__.__name__
        params = {}
        
        for key, value in tfm.__dict__.items():
            if not key.startswith('_') and key != '__signature__':
                if hasattr(value, '__dict__') and hasattr(value, 'target_shape'):
                    params['target_shape'] = value.target_shape
                elif hasattr(value, '__dict__') and not key.startswith('_'):
                    nested_params = {k: v for k, v in value.__dict__.items() 
                                   if not k.startswith('_') and isinstance(v, (int, float, str, bool, tuple, list))}
                    params.update(nested_params)
                elif isinstance(value, (int, float, str, bool, tuple, list)):
                    params[key] = value
        
        return {
            'name': class_name,
            'params': params
        }
        
    def _build_config(self) -> dict[str, Any]:
        """Build configuration dictionary from initialization parameters."""
        # Extract detailed transform information
        transform_details = [self.extract_all_params(tfm) for tfm in self.item_tfms]
        
        return {
            "model_name": self.model_name,
            "loss_function": self.loss_function,
            "transform_details": transform_details,
            "size": self.size,
            "resample": self.resample,
            "reorder": self.reorder,
        }
    
    def _extract_training_params(self) -> dict[str, Any]:
        """Extract training hyperparameters from the learner."""
        params = {}
        
        params["epochs"] = self.learn.n_epoch
        params["learning_rate"] = float(self.learn.lr)
        params["optimizer"] = self.learn.opt_func.__name__
        params["batch_size"] = self.learn.dls.bs
        
        params["loss_function"] = self.config["loss_function"]
        params["size"] = self.config["size"]
        params["resample"] = self.config["resample"]
        params["reorder"] = self.config["reorder"]
        
        params["transformations"] = json.dumps(
            self.config["transform_details"], 
            indent=2, 
            separators=(',', ': ')
        )
        
        return params
    
    def _extract_epoch_metrics(self) -> dict[str, float]:
        """Extract metrics from the current epoch."""
        recorder = self.learn.recorder
        
        # Get custom metric names and values (skip 'epoch' and 'time')
        metric_names = recorder.metric_names[2:]
        raw_metric_values = recorder.log[2:]
        
        metrics = {}
        
        # Process each metric, handling both scalars and tensors
        for name, val in zip(metric_names, raw_metric_values):
            if isinstance(val, torch.Tensor):
                if val.numel() == 1:
                    # Single value tensor (like binary dice score)
                    metrics[name] = float(val)
                else:
                    # Multi-element tensor (like multiclass dice scores)
                    val_list = val.tolist() if hasattr(val, 'tolist') else list(val)
                    # Log individual class scores
                    for i, class_score in enumerate(val_list):
                        metrics[f"{name}_class_{i+1}"] = float(class_score)
                    # Log mean across classes
                    metrics[f"{name}_mean"] = float(torch.mean(val))
            else:
                metrics[name] = float(val)
        
        # Handle loss values
        if len(recorder.log) >= 2:
            metrics['train_loss'] = float(recorder.log[1])
            if len(recorder.log) >= 3:
                metrics['valid_loss'] = float(recorder.log[2])
                
        return metrics
    
    def _save_model_artifacts(self, temp_dir: Path) -> None:
        """Save model weights, learner, and configuration as artifacts."""
        weights_path = temp_dir / "weights"
        self.learn.save(str(weights_path))
        
        weights_file = f"{weights_path}.pth"
        if os.path.exists(weights_file):
            mlflow.log_artifact(weights_file, "model")
        
        learner_path = temp_dir / "learner.pkl"
        self.learn.export(str(learner_path))
        mlflow.log_artifact(str(learner_path), "model")
        
        config_path = temp_dir / "inference_settings.pkl"
        store_variables(config_path, self.size, self.reorder, self.resample)
        mlflow.log_artifact(str(config_path), "config")
    
    def _register_pytorch_model(self) -> None:
        """Register the PyTorch model with MLflow."""
        mlflow.pytorch.log_model(
            pytorch_model=self.learn.model,
            registered_model_name=self.model_name
        )
    
    def before_fit(self) -> None:
        """Log hyperparameters before training starts."""
        params = self._extract_training_params()
        mlflow.log_params(params)
    
    def after_epoch(self) -> None:
        """Log metrics after each epoch."""
        metrics = self._extract_epoch_metrics()
        if metrics:
            mlflow.log_metrics(metrics, step=self.learn.epoch)
    
    def after_fit(self) -> None:
        """Log model artifacts after training completion."""
        print("\nTraining finished. Logging model artifacts to MLflow...")
        
        with tempfile.TemporaryDirectory() as temp_dir:
            temp_path = Path(temp_dir)
            
            self._save_model_artifacts(temp_path)
            
            self._register_pytorch_model()
            
        print(f"MLflow run completed. Run ID: {mlflow.active_run().info.run_id}")